In [1]:
import jieba
import jieba.posseg as pseg
#用于中文分词的库
from py2neo import Graph
#用于中文备注的库

In [2]:
#用于导入打标的文件
def dictPath(dict_path = []):
    for p in dict_path:
        jieba.load_userdict(p)

In [3]:
# 导入打标文件
dictPath(['./Data/ngsDict.txt', './Data/nsxDict.txt','./Data/nprDict.txt'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HP\AppData\Local\Temp\jieba.cache
Loading model cost 0.681 seconds.
Prefix dict has been built successfully.


In [4]:
# 用于分词，把达标过的词从搜索语句中找出来
def get_word_objects(sentence):
    ngs = []
    nsx = []
    npr = []
    for word, tag in pseg.cut(sentence):
        if tag == 'ngs':
            ngs.append(word)
        elif tag == 'nsx':
            nsx.append(word)
        elif tag == 'npr':
            npr.append(word)   
    return ngs, nsx, npr

In [5]:
# 连接neo4j放在文件最前面
url = 'localhost:7474'
# 用户名
username = 'neo4j'
# 密码
password = '990913'
graph = Graph(url, username=username, password=password)

# 通过一句通用的搜索语句返回所有需要的东西
def searchGraph(label, value):
    rt = graph.run(
        "match (s:`{}`)-[p]->(o) where s.value=~'.*{}.*' return type(p), o.value, labels(o)"
        .format(label,value)).data()
    if len(rt) < 1:
        return -1
    else:
        property_dict = {}
        for po in rt:
            if po['type(p)'] not in property_dict:
                property_dict[po['type(p)']] = [(po['labels(o)'][0], po['o.value']), ]
            else:
                property_dict[po['type(p)']].append((po['labels(o)'][0], po['o.value']))
        return property_dict

In [6]:
# 把neo4j中返回的所有关系进行整理，返回需要的数据
def search(sentence):
    ngs, nsx, npr = get_word_objects(sentence)
    if ngs:
        answ = []
        for i in ngs:
            answer = searchGraph(label='理财产品', value=i)
            ans = []
            if nsx:
                for k,v in answer.items():
                    if k in nsx:
                        a = str(k + ' : ' + v[0][1])
                        ans.append(a)
            else:
                for p,q in answer.items():
                    a = p + ' : ' + q[0][1]
                    ans.append(a)
            answ.append(ans)
        return answ
    elif npr:
        anws = []
        for i in npr:
            lab = i.split('为')[0]
            val = i.split('为')[1]
            answer = searchGraph(label=lab, value=val)
            for j in answer['产品']:
                anws.append(j[1])
        return anws
    else:
        return '请输入想搜索的理财产品全名，或者想要找的属性'

In [7]:
s = input()
search(s)

金宝理财2020055期


[['认购价格 : 50000.0',
  '机构类别 : 农村合作金融机构',
  '产品销售区域 : 江苏',
  '收益类型 : 保本浮动收益',
  '实际天数（天） : 90',
  '产品状态 : 预售',
  '产品终止日期 : 2020/12/21',
  '产品起始日期 : 2020/09/22',
  '募集结束日期 : 2020/09/15',
  '募集起始日期 : 2020/09/15',
  '风险等级 : 一级(低)',
  '风险等级编号 : 1',
  '募集币种 : 人民币(CNY)',
  '投资性质 : 固定收益类',
  '运作模式 : 封闭式非净值型',
  '募集方式 : 公募',
  '发行机构代码 : C31324',
  '发行机构 : 徐州铜山农村商业银行股份有限公司',
  '业绩比较基准 : 0',
  '期限类型 : 1-3个月(含)',
  '登记编码 : C3132420000049']]

In [8]:
# 不封装写在main里面的写法
sentence = input()
ngs, nsx, npr = get_word_objects(sentence)
if ngs:
    for i in ngs:
        answer = searchGraph(label='理财产品', value=i)
        if nsx:
            for k,v in answer.items():
                if k in nsx:
                    print(k + ' : ' + v[0][1])
        else:
            for p,q in answer.items():
                print(p + ' : ' + q[0][1])
elif npr:
    for i in npr:
        lab = i.split('为')[0]
        val = i.split('为')[1]
        answer = searchGraph(label=lab, value=val)
        for j in answer['产品']:
            print(j[1])
else:
    print('请输入理财产品全名')

产品销售区域为江苏的产品
汇财之舟理财974号
金陶2024期月月盈理财
金陶2024期双月盈理财
南通农村商业银行金贝财富·共盈理财月月盈-180
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第268期（特定客户专享）理财产品
中国建设银行江苏省分行“乾元添福”2020年第305期人民币理财产品
中国建设银行江苏省分行“乾元添福”2020年第166期人民币理财产品
“金莲花”共享2020年9200102091号理财
金石榴惠盈安鑫享2号18M48
金杨树聚赢添富141期
鑫湖系列第213期180天
天天向上721号
金银杏非保本理财2020年第69期
龙川理财2025期A款
启富东疆添盈理财664期
海门农商银行“江海财富-月安享”系列人民币理财第180期
海门农商银行“江海财富-双季盈”系列人民币理财第180期
海门农商银行“江海聚宝-双季盈”系列人民币理财第127期
金蝶理财2426期
启富东疆臻盈理财A20001期
启富东疆臻盈理财A20002期
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第240期理财产品
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第241期理财产品
金蝶理财2424期
金蝶理财2425期
盐城同鑫聚富·聚农宝2020E037期理财产品
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第41期（端午节专享）理财产品
聚富理财644期
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第244期理财产品
汇富增盈B200109
盐城同鑫聚富·金盈鑫2020D064期理财产品
中国建设银行苏州分行“乾元―鑫溢江南”非保本型人民币2020年第30期理财产品
汇富合赢A20010
中国建设银行江苏省分行“乾元添福”2020年第150期人民币理财产品
2020年工银财富专属理财产品JSFJA058
紫金财富安鑫盈200042期人民币理财产品
中国建设银行江苏省分行“乾元添福”2020年第281期人民币理财产品
中国建设银行江苏省分行“乾元添福”2020年第294期人民币理财产品
南通农村商业银行金贝财富·共盈理财双月盈-180
南通农村商业银行金贝财富·共盈理财季季丰-180
金杨树聚赢添富142期
金陶2024期季季盈理财
金陶2024期双季盈理财
“金莲花”共享2

In [67]:
get_word_objects('实际天数(天)')

([], [], [])